In [1]:
import gdown
import os
import zipfile

In [2]:
url = "https://drive.google.com/file/d/149vtxI7-ZI7BfB-7ZpAfZeFM4UvNU04Y/view?usp=drive_link"

In [3]:
file_id = url.split("/")[-2]
file_id

'149vtxI7-ZI7BfB-7ZpAfZeFM4UvNU04Y'

In [3]:
os.chdir("../")

In [ ]:
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "data/fracture_images.zip")

In [8]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...


In [7]:
with zipfile.ZipFile('./data/fractures.zip', 'r') as zipref:
    zipref.extractall('./data')

In [9]:
import yaml
dataYAMLDir = './data/data.yaml'
with open(dataYAMLDir, 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

num_classes

'7'

In [17]:
with open('./data/data.yaml', 'r') as file:
    contents = file.read()
    print(contents)

train: ./data/train/images
val: ./data/valid/images
test: ./data/test/images

nc: 7
names: ['elbow positive', 'fingers positive', 'forearm fracture', 'humerus fracture', 'humerus', 'shoulder fracture', 'wrist positive']


In [10]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [11]:
%pwd

'c:\\Users\\asort\\OneDrive\\Documents\\AsorBackup\\Learn\\fracture-detection'

In [12]:
%%writetemplate ./yolov5/models/custom_yolov5s.yaml

nc: {num_classes} # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]


In [2]:
%pwd

'c:\\Users\\asort\\OneDrive\\Documents\\AsorBackup\\Learn\\fracture-detection\\research'

In [3]:
os.chdir("../")

In [ ]:
%%time
!python ./yolov5/train.py --img 416 --batch 16 --epochs 50 --data ./data/data.yaml --cfg ./yolov5/models/custom_yolov5s.yaml --weights 'YOLOv5s' --name yolov5s_results --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs